In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Model, load_model
from keras.layers import Conv2D, Input, MaxPooling2D, Dense, Dropout, Flatten
from keras.layers import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
submission = pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv")
X = train.drop(['label'],1).values
Y = train['label'].values
x_test = test.values

X = X/255.
x_test = x_test/255.

X = X.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

Y = to_categorical(Y)


In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(X,Y, test_size=0.1)

In [ ]:
def get_model():
    In = Input(shape=(28,28,1))
    x = Conv2D(32, (3,3), padding="same")(In)
    x = LeakyReLU(alpha=0.01)(x)
    x = Conv2D(32, (3,3), padding="same")(x)
    x = LeakyReLU(alpha=0.01)(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(64, (3,3), padding="same")(x)
    x = LeakyReLU(alpha=0.01)(x)
    x = Conv2D(64, (3,3), padding="same")(x)
    x = LeakyReLU(alpha=0.01)(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)
    x = Dropout(0.2)(x)
    x = Conv2D(128, (3,3), padding="same")(x)
    x = LeakyReLU(alpha=0.01)(x)
    x = Conv2D(128, (3,3), padding="same")(x)
    x = LeakyReLU(alpha=0.01)(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)
    x = Dropout(0.2)(x)
    x = Flatten()(x)
    x = Dense(512)(x)
    x = LeakyReLU(alpha=0.01)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(256)(x)
    x = LeakyReLU(alpha=0.01)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(128)(x)
    x = LeakyReLU(alpha=0.01)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    Out = Dense(10, activation="softmax")(x)
    model = Model(In, Out)
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model
    
model = get_model()
model.summary()

In [ ]:
best_checkpoint = ModelCheckpoint('best.hdf5',monitor = 'val_loss', mode = "min", verbose = 1, save_best_only = True)
lr_reduction = ReduceLROnPlateau(monitor = 'val_loss', patience = 3, verbose = 1, factor = 0.5, min_lr = 1e-6)

In [ ]:
data_generator = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


epochs = 100
batch_size = 128

train_generator = data_generator.flow(x_train, y_train, batch_size=batch_size)
valid_generator = data_generator.flow(x_valid, y_valid, batch_size=batch_size)

In [ ]:
hist = model.fit_generator(train_generator, epochs=epochs, steps_per_epoch = x_train.shape[0]//batch_size,
                    validation_data = valid_generator, validation_steps = x_valid.shape[0]//batch_size, callbacks=[best_checkpoint, lr_reduction], verbose=1)

In [ ]:
plt.plot(hist.history["val_loss"])
plt.plot(hist.history["loss"])
plt.legend(["val_loss","loss"])
plt.show()
plt.plot(hist.history["val_accuracy"])
plt.plot(hist.history["accuracy"])
plt.legend(["val_accuracy","accuracy"])
plt.show()

In [ ]:
best = load_model("best.hdf5")
preds = best.predict(x_test, verbose=1)
preds = np.array([np.argmax(i) for i in preds])
preds

In [ ]:
submission['Label'] = preds
submission.to_csv("submission.csv", index=False)
submission.head()